In [1]:
import os
os.chdir("../")
%pwd

'/home/malik-harris/tekhqs/didenv/Decentralized_Identity_Verification'

In [ ]:
model_path = "Model/best_kyc_siamese.onnx"
settings_path = "zk/settings.json"
compiled_model_path = "zk/network.compiled"
pk_path = "zk/key.pk"
vk_path = "zk/key.vk"
witness_path = "zk/witness.json"
data_path = "zk/input.json"
os.makedirs("zk", exist_ok=True)

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import os
import json

# --- Paths
inference_dir = "inference"
doc_img_path = None
selfie_img_path = None

# --- Find one doc + selfie image pair
for file in os.listdir(inference_dir):
    if any(x in file.lower() for x in ["id", "passport", "national"]) and doc_img_path is None:
        doc_img_path = os.path.join(inference_dir, file)
    elif "selfie" in file.lower() and selfie_img_path is None:
        selfie_img_path = os.path.join(inference_dir, file)

assert doc_img_path and selfie_img_path, "No valid doc/selfie image found."

# --- Same transform as training
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# --- Load & transform
doc_tensor = transform(Image.open(doc_img_path).convert("RGB")).unsqueeze(0)  # shape: [1, 3, 112, 112]
selfie_tensor = transform(Image.open(selfie_img_path).convert("RGB")).unsqueeze(0)

# --- Flatten tensors to 1D lists
doc_array = doc_tensor.flatten().tolist()
selfie_array = selfie_tensor.flatten().tolist()

# --- Format for EZKL
data = {
    "input_data": [doc_array, selfie_array]
}

# --- Save to JSON
with open("zk/input.json", "w") as f:
    json.dump(data, f)


print(f"✅ Saved input to zk/input.json:\nDoc: {doc_img_path}\nSelfie: {selfie_img_path}")

In [ ]:
# import torch
# import json

# # Create dummy input tensors
# doc_tensor = torch.rand(1, 3, 224, 224)
# selfie_tensor = torch.rand(1, 3, 224, 224)

# # Flatten and convert tensors to lists
# doc_array = doc_tensor.flatten().tolist()
# selfie_array = selfie_tensor.flatten().tolist()

# # Prepare the input data dictionary
# data = {
#     "input_data": [doc_array, selfie_array]
# }

# # Save to a JSON file
# with open("zk/input.json", "w") as f:
#     json.dump(data, f)

In [ ]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
session = ort.InferenceSession("Model/best_kyc_siamese.onnx")

# Prepare dummy inputs
inputs = {
    "doc_img": np.random.rand(1, 3, 112, 112).astype(np.float32),
    "selfie_img": np.random.rand(1, 3, 112, 112).astype(np.float32)
}

# Run a forward pass
outputs = session.run(None, inputs)
print("Sigmoid output (probability):", outputs)


In [3]:
import ezkl

In [4]:
run_args = ezkl.PyRunArgs()
run_args.input_visibility = "private"       
run_args.param_visibility = "private"         
run_args.output_visibility = "public"       
run_args.num_inner_cols = 2                 # Leave at 2 unless you have perf issues
run_args.logrows = 16
run_args.input_scale = 8                
run_args.param_scale = 8
run_args.variables = [("batch_size", 1)] 

In [ ]:
res = ezkl.gen_settings(model_path, settings_path,py_run_args=run_args)
assert res == True

In [ ]:
res = await ezkl.calibrate_settings(data_path, model_path, settings_path, target="resources", scales = [1, 2, 4, 8, 16] )
assert res == True
print("verified")

In [ ]:
ezkl.compile_circuit(model_path, compiled_model_path, settings_path)

In [ ]:
res = await ezkl.get_srs(settings_path)
print("SRS generated:", res)

In [ ]:
res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
print("Witness generated:", res)

In [5]:
!export ENABLE_ICICLE_GPU=true
!export ICICLE_SMALL_K=7

In [6]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

print("Setup completed:", res)

dynamic lookup has 3 blocks
dynamic lookup has 3 blocks
dynamic lookup has 3 blocks


: 